In [1]:
import numpy as np
import pymongo

client = pymongo.MongoClient()
db = client['espace5']
forum_messages = db['forumsbis']

In [2]:
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from nltk.tokenize import WordPunctTokenizer

fr_textblob = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer(), tokenizer=WordPunctTokenizer())

In [3]:
import re
token_pattern = re.compile(r"(?u)\b\w\w+\b")

all_text = " ".join([message["text"] for message in forum_messages.find()])
all_text = fr_textblob(" ".join(token_pattern.findall(all_text)))
print all_text.sentiment

(0.08251793230515159, 0.27991056878286)


In [4]:
from nltk.stem.snowball import FrenchStemmer
from nltk.corpus import stopwords

stemmer = FrenchStemmer()

stop = stopwords.words('french')
stop += ["http", "https", "renault", "espace"]
stop += ['les', 'a', 'plus', u'ça', 'ils', 'cette', 'car', 'de']

In [8]:
len_text_mini = 0

raw_texts = []
user_ids = []
mdates = []
forum_ids = []

for message in forum_messages.find():
    text = message["text"]
    text = text.replace(u'Reprise du message précédent :', "")
    text = text.replace('  ', ' ').replace('\t', ' ').replace('\n', ' ').replace('\r', ' ')
    text = text.strip().lstrip(' ')
    if len(text) > len_text_mini:
        raw_texts.append(text)
        user_ids.append(message["user"])
        mdates.append(message["date"])
        forum_ids.append(message["forum_id"])
        
nbr_messages = len(raw_texts)

texts_cleaned = [' '.join([token.lower() for token in token_pattern.findall(text) if token.lower() not in stop]) \
                 for text in raw_texts]

texts_stemmed = [' '.join([stemmer.stem(token) for token in fr_textblob(text).tokens]) \
                 for text in texts_cleaned]

texts_nouns = [' '.join([tag[0] for tag in fr_textblob(text).tags if tag[1] in [u'NN', u'NNS', u'NNP', u'VBN']]) \
               for text in texts_cleaned]

###Agglomerative Clustering and PCA

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA

In [10]:
st_word_vectorizer = TfidfVectorizer(ngram_range=(1, 3), strip_accents='unicode', norm='l2',
                                     min_df=5, token_pattern=u'(?u)\\b\\w\\w+\\b')
stemmed_text_vectors = st_word_vectorizer.fit_transform(texts_stemmed)
print stemmed_text_vectors.shape

(1062, 1171)


/usr/local/lib/python2.7/dist-packages/numpy/core/fromnumeric.py:2507: VisibleDeprecationWarning: `rank` is deprecated; use the `ndim` attribute or function instead. To find the rank of a matrix see `numpy.linalg.matrix_rank`.
  VisibleDeprecationWarning)


In [11]:
nbr_clusters = 12

In [12]:
reducer = PCA(n_components=227)
reduced_X = reducer.fit_transform(stemmed_text_vectors.toarray())
reduced_X.shape

(1062, 227)

In [13]:
agcr = AgglomerativeClustering(linkage='average', affinity='cosine', n_clusters=nbr_clusters)
Yagcr = agcr.fit_predict(reduced_X)

d_agcr_clusters = {}
for i in range(nbr_messages):
    if Yagcr[i] not in d_agcr_clusters:
        d_agcr_clusters[Yagcr[i]] = []
    else:
        d_agcr_clusters[Yagcr[i]].append(i)

In [14]:
for i in range(nbr_clusters):
    print "Cluster ", i, ":", len(d_agcr_clusters[i]), 'similar statements'

Cluster  0 : 247 similar statements
Cluster  1 : 119 similar statements
Cluster  2 : 104 similar statements
Cluster  3 : 43 similar statements
Cluster  4 : 54 similar statements
Cluster  5 : 43 similar statements
Cluster  6 : 35 similar statements
Cluster  7 : 88 similar statements
Cluster  8 : 69 similar statements
Cluster  9 : 48 similar statements
Cluster  10 : 88 similar statements
Cluster  11 : 112 similar statements


In [15]:
from sklearn.metrics import pairwise_distances

agcr_cluster_nouns_sentences = [" ".join([texts_nouns[r] for r in d_agcr_clusters[i]]) for i in range(nbr_clusters)]
agcr_cluster_noun_vectorizer = TfidfVectorizer()
agcr_cluster_nouns_vectors = agcr_cluster_noun_vectorizer.fit_transform(agcr_cluster_nouns_sentences)
    
for i in range(nbr_clusters):
    r = agcr_cluster_nouns_vectors.getrow(i)
    n = len(r.indices)
    s = zip(np.array(agcr_cluster_noun_vectorizer.get_feature_names())[r.indices], r.data)
    sorted_s = sorted(s, key=lambda item: item[1], reverse=True)
    print "\n******** cluster: ", i, " with ", len(d_agcr_clusters[i]), 'similar statements'
    print "Key words: "+" ".join([sorted_s[x][0].upper() for x in range(n)][:15])

    cluster_center = np.mean(reducer.transform(stemmed_text_vectors[d_agcr_clusters[i]].toarray()), axis=0)
    dist2center = []
    for j in d_agcr_clusters[i]:
        dist2center.append(pairwise_distances(cluster_center, reducer.transform(stemmed_text_vectors[j].toarray()), metric='cosine')[0][0])
    s = zip([str(mdates[r])[:10] for r in d_agcr_clusters[i]], dist2center, [raw_texts[r] for r in d_agcr_clusters[i]])
    sorted_s = sorted(s, key=lambda item: item[1])
    count_print = 0
    for m in sorted_s:
        if len(m[2])>500 and len(m[2])<7500:
            print "\n>>> Message date: %10s (%.3f)\n   %15s" % m
            count_print += 1
            if count_print == 3:
                break
    print '---'


******** cluster:  0  with  247 similar statements
Key words: MERCI POUCES TAXE ZEN CONTROL EUROS LOA PACK FRANCE INFO VILLE LLD INTENS OPTION QUELQU

>>> Message date: 2015-03-26 (0.806)
   non en France il y a un modele de plus nous on a une Zen , intens et initiale mais par exemple l'intens est moins chere chez nous mais avec moins d'options (pas de cuir par defaut) Mais la différence de modèle n'y fait rien sur les taxes que l'on doit payer sur les voitures, en Belgique il risque de vendre plus de 130 car il y a une différence de taxe de 744 euros entre le DCI130 et le DCI160 pour le meme moteur!. Y'a aussi des tucs idiots , par exemple l'ambiance de couleur intérieur disparait quand on prend la boite manuel , le bose en option n'est pas possible sur la version Zen il faut prendre l'intens ...

>>> Message date: 2015-03-26 (0.807)
   MEs sources j'etudie l'achat d'une nouvelle voiture , je vis en Belgique et donc j'ai le tableau des taxes par KW et par CV , j'hesitais a la prendre

###Gensim

In [16]:
from gensim import corpora, models, similarities, matutils 
from gensim.models import LdaModel

In [17]:
documents = texts_stemmed
texts = [document.split() for document in documents]
bigrams = [ [" ".join((u,v)) for (u,v) in zip(text[:-1], text[1:])] for text in texts]
trigrams = [ [" ".join((u,v,w)) for (u,v,w) in zip(text[:-1], text[1:], text[2:])] for text in texts]

texts = [ u+v+w for (u,v,w) in zip(texts, bigrams, trigrams) ]

In [18]:
all_tokens = sum(texts, [])
tokens_5 = set(word for word in set(all_tokens) if all_tokens.count(word) < 5)
dtexts = [[word for word in text if word not in tokens_5] for text in texts]

dictionary = corpora.Dictionary(dtexts)

In [19]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
tfidf = models.TfidfModel(corpus, id2word=dictionary)
tfidfcorpus = tfidf[corpus]

In [ ]:
tlda = LdaModel(tfidfcorpus, num_topics=nbr_clusters, id2word=dictionary, update_every=0, passes=100)

In [ ]:
reduced_X = matutils.corpus2dense(tlda[tfidfcorpus], nbr_clusters).T

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agcr = AgglomerativeClustering(linkage='average', affinity='cosine', n_clusters=nbr_clusters)
Yagcr = agcr.fit_predict(reduced_X)

d_agcr_clusters = {}
for i in range(nbr_messages):
    if Yagcr[i] not in d_agcr_clusters:
        d_agcr_clusters[Yagcr[i]] = []
    else:
        d_agcr_clusters[Yagcr[i]].append(i)

In [31]:
for i in range(nbr_clusters):
    print "Cluster ", i, ":", len(d_agcr_clusters[i]), 'similar statements'

Cluster  0 : 179 similar statements
Cluster  1 : 41 similar statements
Cluster  2 : 82 similar statements
Cluster  3 : 48 similar statements
Cluster  4 : 109 similar statements
Cluster  5 : 119 similar statements
Cluster  6 : 63 similar statements
Cluster  7 : 47 similar statements
Cluster  8 : 90 similar statements
Cluster  9 : 84 similar statements
Cluster  10 : 144 similar statements
Cluster  11 : 44 similar statements


In [32]:
agcr_cluster_nouns_sentences = [" ".join([texts_nouns[r] for r in d_agcr_clusters[i]]) for i in range(nbr_clusters)]

agcr_cluster_noun_vectorizer = TfidfVectorizer()
agcr_cluster_nouns_vectors = agcr_cluster_noun_vectorizer.fit_transform(agcr_cluster_nouns_sentences)

for i in range(nbr_clusters):
    r = agcr_cluster_nouns_vectors.getrow(i)
    n = len(r.indices)
    s = zip(np.array(agcr_cluster_noun_vectorizer.get_feature_names())[r.indices], r.data)
    sorted_s = sorted(s, key=lambda item: item[1], reverse=True)
    print "\n******** cluster: ", i, " with ", len(d_agcr_clusters[i]), 'similar statements'

    print "Key words: "+" ".join([sorted_s[x][0].upper() for x in range(n)][:15])
    
    cluster_center = np.mean(reduced_X[d_agcr_clusters[i]], axis=0)
    dist2center = []
    for j in d_agcr_clusters[i]:
        dist2center.append(pairwise_distances(cluster_center, reduced_X[j], metric='cosine')[0][0])
    s = zip([str(mdates[r])[:10] for r in d_agcr_clusters[i]], dist2center, [raw_texts[r] for r in d_agcr_clusters[i]])
    sorted_s = sorted(s, key=lambda item: item[1])
    count_print = 0
    for m in sorted_s:
        if len(m[2])>200 and len(m[2])<7500:
            print "\n>>> Message date: %10s (%.3f)\n   %15s" % m
            count_print += 1
            if count_print == 1:
                break
    print '---'


******** cluster:  0  with  179 similar statements
Key words: VOTE GAMME FONCÉ TAXE VÉHICULE ĘTRE PRIX LIVRAISON MISE CONCESSION COULEUR DESIGN PLACE EUROS PLACES

>>> Message date: 2015-03-17 (0.021)
   J'ai demandé à mon agence ce tantôt pour une simulation d'Initiale, que ce soit essence ou diesel nous sommes à fin Juillet en délai de livraison, le 29 exactement. Pour ceux que ça intéressera (je ne vais pas la poster pour avoir des réflexions, à me demander en MP) j'ai la documentation complète avec les références et les tarifs des accessoires (l'attelage en fait partie), comme quoi tous les collaborateurs ne sont pas à mettre au même rang, pour ceux qui les aiment pas.
---

******** cluster:  1  with  41 similar statements
Key words: BORD PLANCHE GRIS LUNE BRUN PHOTO TABLEAU SELLERIE ZOE REFLETS REFLET CASSIOPÉE FONÇÉ INAUGURATION ULLPDF

>>> Message date: 2015-04-10 (0.139)
   D'ailleurs je peux confirmer la garantie 5 ans jusqu'en septembre, c'est réel et pour tout le monde. De 

In [33]:
average_mlength = len(all_tokens)/nbr_messages
print average_mlength

66


In [34]:
ltfidfcorpus = list(tfidfcorpus)
tlda_topic_count = [sorted(tlda[ltfidfcorpus[m]], key=lambda item: item[1], reverse=True)[0][0] for m in range(nbr_messages)]
tlda_topic_count = dict((i,tlda_topic_count.count(i)) for i in tlda_topic_count)
print tlda_topic_count

tlda_topic_sigword_count = [len([x for x in tlda.show_topic(i, 100) if x[0]>0.0033]) for i in range(nbr_clusters)]
print tlda_topic_sigword_count

{0: 95, 1: 87, 2: 129, 3: 166, 4: 61, 5: 50, 6: 93, 7: 144, 8: 47, 9: 44, 10: 35, 11: 111}
[67, 61, 77, 71, 71, 78, 73, 75, 62, 58, 53, 64]


In [35]:
for i in range(nbr_clusters):
    print "Topic", i, " ".join(zip(*sorted([(  (k[0]+np.mean(zip(*tlda.show_topic(i, tlda_topic_sigword_count[i]))[0]))/
                                               (all_tokens.count(k[1])+average_mlength)       , k[1])
                                            for k in tlda.show_topic(i, tlda_topic_sigword_count[i])],
                                    reverse=True)[:12])[1]).upper()

Topic 0 RETARD CHANG SOURC DOMMAG RIEN TDI CV DÉSOL HABIT CENTRAL VIEUX PER
Topic 1 MERC MOTO AUTO MOTO MERC INFO OK MERC OK ESSAI AUTO MOTO ESSAI AUTO HDG SER PRÉ DOIT ÊTRE
Topic 2 19 MOD SÉQUENTIEL MOD SÉQUENTIEL POUC 20 CONTROL ENTRE HÉSIT LOOK 19 POUC FR ACTUALIT
Topic 3 COULEUR FONC PHOTOS TAX INTÉRIEUR BLANC 18 BEL BONSOIR ICI ACCESSOIR TANT
Topic 4 YOUTUB YOUTUB COM WATCH YOUTUB COM WWW YOUTUB COM WWW YOUTUB COM WATCH TEST WATCH FÉLICIT 80 SÛR 80 120
Topic 5 GRAND IV VITESS MODIF VILL 100 04 PUISSANC VALEUR SURTOUT TRAFIC 30 RAPPORT POID PUISSANC
Topic 6 ESSAIS PRESS ETRE PRESS COUT PEUT ÊTRE RAPPEL CHANC VENT SOUVENT MODIFI PEUT ETRE LANGU
Topic 7 CON DÉJÀ SECOUR COMMERCIALIS ROU SECOUR CARADISIAC COM AVIS CONSTRUCTEUR 110 13 CAS ENTRÉ
Topic 8 CAR PANNEAU 1700 VIDEO ALLEMAND FUTUR ELLE FR MOCH CI ZEN REVENT
Topic 9 BESOIN FREIN SAIS BIENTÔT REGRET INFORM SUFF DISPONIBL MAGAZIN LAISS PARKING SAIT
Topic 10 COOL GRIS BORD TOUT MOND LUN PLANCH BORD GRIS LUN BORD GRIS BRUN BORD GRIS

In [36]:
from datetime import datetime

d_gensim_topics = {}
for m in range(nbr_messages):
    s = sorted(tlda[ltfidfcorpus[m]], key=lambda item: item[1], reverse=True)[0]
    if s[1] > 0.5 and mdates[m] > datetime(2014,10,1) :
        if s[0] not in d_gensim_topics:
            d_gensim_topics[s[0]] = []
        toplist = len(raw_texts[m])*(1000-len(raw_texts[m]))
        d_gensim_topics[s[0]].append((m, s[1], toplist))

In [37]:
for x in [(k, sorted(d_gensim_topics[k], key=lambda item: item[2], reverse=True)[:5]) for k in d_gensim_topics.keys()]:
    print ">>>>>>>> Topic", x[0], "with", tlda_topic_count[x[0]], "similar statements:"
    print "[KEY WORDS]", " ".join(zip(*sorted([(  (k[0]+np.mean(zip(*tlda.show_topic(i, tlda_topic_sigword_count[x[0]]))[0]))/
                                                  (all_tokens.count(k[1])+average_mlength), k[1])
                                               for k in tlda.show_topic(x[0], tlda_topic_sigword_count[x[0]])],
                                       reverse=True)[:12])[1]).upper()
    print
    for y in x[1]:
        print "*** "+str(mdates[y[0]])+" : "+raw_texts[y[0]]
    print

>>>>>>>> Topic 0 with 95 similar statements:
[KEY WORDS] RETARD CHANG SOURC DOMMAG RIEN CV TDI DÉSOL CENTRAL HABIT VIEUX PER

*** 2015-03-07 09:55:36 : Je te rejoins parfaitement j2c sur le fait que ce nouvel espace n'est pas vraiment un espace.      Par contre je comprends qu'ils ait gardé le nom dans la mesure ou ce nom ŕ une image forte (pendant de nombreuses années Espace était associé ŕ Monospace et on parlait d'Espace citroen ou peugeot ou VW...) et ou il n'y a pas d'autres véhicules prévus ŕ męme de remplacer l'espace.      J'ai été moi aussi bercé par l'espace dans mon enfance, j'en ai révé pendant des années et on ŕ fini par en avoir un en 98
*** 2015-04-01 15:28:18 : Je viens de recevoir le mag aujourd'hui (merci l'abonnement qui fait qu'il arrive toujours après sa sortie en kiosque  ) Effectivement dans l'essai du dCi 160 edc il est noté que le moteur est mollasson pour déplacer la masse du véhicule...vont être content ceux qui avaient un dCi 175  Erik123 a écrit : Je pense 